#TRATAMENTO DE DADOS
# Para juntar duas tabelas -> pd.merge()
# Para transformar colunas em linhas -> pd.melt()
# Para transformar linhas em colunas -> pd.pivot()


In [1]:
import pandas as pd
import numpy as np
import psycopg2 as ps
import matplotlib.pyplot as plt

conn = ps.connect(host='localhost',
                    database='Projeto',
                    user='postgres',
                    password='3273781620',
                    port=5432)



In [2]:
def columns_treatment(df):
    df.columns = [x.upper() for x in df.columns]
    df.columns = [x.replace(' ', '_') for x in df.columns]
    
    df.columns = [x.replace('/', '_') for x in df.columns]
    df.columns = [x.replace(',', '_') for x in df.columns]
    df.columns = [x.replace('-', '_') for x in df.columns]
    df.columns = [x.replace("'", '') for x in df.columns]
    
        
        
    
    return df

def insert_into_table(conn, df, tablename):
    curr = conn.cursor()
    try:
        

        command = f"""INSERT INTO {tablename} (""" 
        final_part =''

        for idx in df.columns:
            command = command + idx + ', '
            final_part = final_part +'%s,'

        command = command[:-2] + ') VALUES(' + final_part[:-1] + ')'

        insert_into_videos = (command)


        for i, row in df.iterrows():
            row_to_insert = []

            for column in df.columns:
                row_to_insert = row_to_insert + [row[column]]

            curr.execute(insert_into_videos, row_to_insert)
            
    except Exception as e:
        print(f'FALHA NO PREENCHIMENTO DA TABELA: {e}')
        curr.execute("ROLLBACK")
    conn.commit()
    

    
def create_table(df, tablename,  conn):
    
    command = f"""CREATE TABLE {tablename} ( """

    for idx in df.columns:
        command = command+idx+' '
        if pd.api.types.is_integer_dtype(df[idx]):
            command = command+'INT,'
        elif pd.api.types.is_float_dtype(df[idx]):
            command = command+'FLOAT,'
        else:
            command = command+'VARCHAR,'
    command = command[:-1]        
    command = command + ' )'
    
    
    curr = conn.cursor()
    try:
        curr.execute(f'DROP TABLE IF EXISTS {tablename}')
        create_table_command = (command)

        curr.execute(create_table_command)
    except Exception as e:
        print(f'FALHA NA CRIAÇÃO DA TABELA: {e}')
        curr.execute("ROLLBACK")
    conn.commit()
    
    insert_into_table(conn, df,tablename)
    
    

def mg(query,conn,df):
    #query = "SELECT * FROM ADULT_MORTALITY;"
    
    
    df1 = pd.read_sql(query, conn)
    
    try:
        df1=df1[df1['sex']=='Both sexes']
        df1=df1.drop('sex',axis=1)
    except: pass
        
    
    try:
        df_final = df.merge(df1, left_on=['country_name','country_code','year'], right_on=['country_name','country_code','year'],how='left')
    except:
        df_final = df.merge(df1, left_on=['country_name','year'], right_on=['country_name','year'],how='left')

    try:
        df_final.drop(['region_code', 'region'],axis=1,inplace=1)
    except:
        pass
    
    return df_final

In [13]:
# Loading the data LIFE EXPECTANCY
df = pd.read_csv('dataset\Life_expectancy_at_birth_total_years.csv')

# melting the table
df = pd.melt(df.drop(['Indicator Code','Indicator Name'],axis=1), id_vars=['Country Name','Country Code'], value_vars=None, var_name='YEAR', value_name='LIFE_EXPECTANCY', col_level=None, ignore_index=True)

# making all the columns names with Upper case
columns_treatment(df)

#dropping nan values
df.dropna(inplace=True)

# changing the type of columns 
df = df.astype({"YEAR": int})

print(df.dtypes)



create_table(df=df,tablename='LIFE_EXPECTANCY',conn=conn )
df

COUNTRY_NAME        object
COUNTRY_CODE        object
YEAR                 int32
LIFE_EXPECTANCY    float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,LIFE_EXPECTANCY
0,Aruba,ABW,1960,65.662000
1,Africa Eastern and Southern,AFE,1960,42.716053
2,Afghanistan,AFG,1960,32.446000
3,Africa Western and Central,AFW,1960,37.205380
4,Angola,AGO,1960,37.524000
...,...,...,...,...
15955,Kosovo,XKX,2019,72.495122
15956,"Yemen, Rep.",YEM,2019,66.125000
15957,South Africa,ZAF,2019,64.131000
15958,Zambia,ZMB,2019,63.886000


In [14]:
# Loading the data GDP
df = pd.read_csv('dataset\GDP.csv')

# drop das colunas que nao queremos
df = df.drop(['Indicator Code','Indicator Name'], axis=1)

# melt
df = pd.melt(df, id_vars=['Country Name','Country Code'], value_vars=None, var_name='YEAR', value_name='GDP per capita', col_level=None, ignore_index=True)

# drop nan
df = df.dropna()

# tratando nome das colunas
df = columns_treatment(df)

# verificando os tipos de variavel de cada coluna
print(df.dtypes)

# Mudando a coluna ano para INT
df = df.astype({"YEAR": int})

# verificando os tipos de variavel de cada coluna
print(df.dtypes)

# salvando 
create_table(df=df,tablename='GDP',conn=conn )

df

COUNTRY_NAME       object
COUNTRY_CODE       object
YEAR               object
GDP_PER_CAPITA    float64
dtype: object
COUNTRY_NAME       object
COUNTRY_CODE       object
YEAR                int32
GDP_PER_CAPITA    float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,GDP_PER_CAPITA
1,Africa Eastern and Southern,AFE,1960,147.507808
2,Afghanistan,AFG,1960,59.773234
3,Africa Western and Central,AFW,1960,107.932233
13,Australia,AUS,1960,1810.619230
14,Austria,AUT,1960,935.460427
...,...,...,...,...
16220,Samoa,WSM,2020,4067.843459
16221,Kosovo,XKX,2020,4346.637931
16223,South Africa,ZAF,2020,5655.867654
16224,Zambia,ZMB,2020,985.132436


In [15]:
# Loading the data Children overweight
df = pd.read_csv('dataset\Children-who-are-overweight-sdgs.csv')

df.rename(columns={'Indicator 2.2.2b: Prevalence of overweight in children aged 2-4 (%) - Past - Unscaled':'overweight in children aged 2 to 4 percentage'},inplace=True)

df = df.rename(columns=({'Entity':'Country_name','Code':'Country_code'}) )

# drop nan
df = df.dropna()

# tratando nome das colunas
df = columns_treatment(df)


# verificando os tipos de variavel de cada coluna
print(df.dtypes)


# salvando 
create_table(df=df,tablename='OVERWEIGHT_IN_CHILDREN',conn=conn )
 


df

COUNTRY_NAME                                      object
COUNTRY_CODE                                      object
YEAR                                               int64
OVERWEIGHT_IN_CHILDREN_AGED_2_TO_4_PERCENTAGE    float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,OVERWEIGHT_IN_CHILDREN_AGED_2_TO_4_PERCENTAGE
0,Afghanistan,AFG,1990,20.641
1,Afghanistan,AFG,1995,17.523
2,Afghanistan,AFG,2000,14.942
3,Afghanistan,AFG,2005,15.734
4,Afghanistan,AFG,2010,18.922
...,...,...,...,...
1123,Zimbabwe,ZWE,1995,15.855
1124,Zimbabwe,ZWE,2000,18.075
1125,Zimbabwe,ZWE,2005,16.103
1126,Zimbabwe,ZWE,2010,12.390


In [26]:
# Loading the data Adult Mortality
df = pd.read_csv('dataset\Adult_mortality_rate.csv')
#df = df[df['Indicator']=='probability of dying between 15 and 60 years per 1000 population']

df = df[['Location', 'SpatialDimValueCode','Period','Value']]

df = df.rename(columns={'Location': 'country name', 'SpatialDimValueCode':'country code','Period':'year','Value':'Adult mortality rate probability of dying between 15 and 60 years per 1000 population'})

# drop nan
df = df.dropna()

# tratando nome das colunas
df = columns_treatment(df)

# verificando os tipos de variavel de cada coluna
print(df.dtypes)

# salvando 
create_table(df=df,tablename='ADULT_MORTALITY',conn=conn )

df[df['COUNTRY_NAME']=='Brazil']

df[df['Location']=='Brazil']['Indicator']

89      Adult mortality rate (probability of dying bet...
179     Adult mortality rate (probability of dying bet...
518     Adult mortality rate (probability of dying bet...
642     Adult mortality rate (probability of dying bet...
735     Adult mortality rate (probability of dying bet...
1073    Adult mortality rate (probability of dying bet...
1192    Adult mortality rate (probability of dying bet...
1279    Adult mortality rate (probability of dying bet...
1620    Adult mortality rate (probability of dying bet...
1750    Adult mortality rate (probability of dying bet...
1837    Adult mortality rate (probability of dying bet...
2182    Adult mortality rate (probability of dying bet...
2303    Adult mortality rate (probability of dying bet...
2395    Adult mortality rate (probability of dying bet...
2735    Adult mortality rate (probability of dying bet...
2862    Adult mortality rate (probability of dying bet...
2951    Adult mortality rate (probability of dying bet...
3292    Adult 

In [17]:
# Loading the data Daily  Supply of Calories
df = pd.read_csv('dataset\Daily supply of calories _ adult-men-overweight-or-obese-vs-daily-supply-of-calories.csv')

df.drop(['Population (historical estimates)','Continent'],axis=1,inplace=True)

df = df.rename(columns={'Entity':'Country name','Code':'Country Code','Daily caloric supply (FAO (2017) & Various historical sources)':'Daily caloric supply','Overweight or Obese (NCDRisC (2017))':'Overweight or Obese'})

# drop nan
df = df.dropna()

# tratando nome das colunas
df = columns_treatment(df)

df = df[df['YEAR']>1960]

# verificando os tipos de variavel de cada coluna
print(df.dtypes)

# salvando 
create_table(df=df,tablename='DAILY_SUPPLY_OF_CALORIES',conn=conn )


df

COUNTRY_NAME             object
COUNTRY_CODE             object
YEAR                      int64
DAILY_CALORIC_SUPPLY    float64
OVERWEIGHT_OR_OBESE     float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,DAILY_CALORIC_SUPPLY,OVERWEIGHT_OR_OBESE
15,Afghanistan,AFG,1975,2752.0,7.1
16,Afghanistan,AFG,1976,2824.0,7.1
17,Afghanistan,AFG,1977,2489.0,7.2
18,Afghanistan,AFG,1978,2621.0,7.3
19,Afghanistan,AFG,1979,2621.0,7.3
...,...,...,...,...,...
55854,Zimbabwe,ZWE,2009,2147.0,17.8
55855,Zimbabwe,ZWE,2010,2168.0,18.1
55856,Zimbabwe,ZWE,2011,2200.0,18.4
55857,Zimbabwe,ZWE,2012,2197.0,18.7


In [18]:
# Loading the data Access to Electricity
df = pd.read_csv('dataset\electry.csv')

# melting the table
df = pd.melt(df.drop(['Indicator Code','Indicator Name'],axis=1), id_vars=['Country Name','Country Code'], value_vars=None, var_name='YEAR', value_name='access_electricity', col_level=None, ignore_index=True)

# making all the columns names with Upper case
columns_treatment(df)

#dropping nan values
df.dropna(inplace=True)

# changing the type of columns 
df = df.astype({"YEAR": int})

print(df.dtypes)
df.head(100)

create_table(df=df,tablename='ELECTRICITY',conn=conn )
df

COUNTRY_NAME           object
COUNTRY_CODE           object
YEAR                    int32
ACCESS_ELECTRICITY    float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,ACCESS_ELECTRICITY
7980,Aruba,ABW,1990,100.000000
7985,Albania,ALB,1990,100.000000
7986,Andorra,AND,1990,100.000000
7988,United Arab Emirates,ARE,1990,100.000000
7989,Argentina,ARG,1990,92.154800
...,...,...,...,...
15955,Kosovo,XKX,2019,100.000000
15956,"Yemen, Rep.",YEM,2019,72.751701
15957,South Africa,ZAF,2019,85.000000
15958,Zambia,ZMB,2019,43.000000


In [19]:
# Loading the data OBESITY
df = pd.read_csv('dataset\obesity.csv')

# Eliminating first column and renaming 
df = df.iloc[: , 1:]
df = df.rename(columns={'Country':'Country name','Obesity (%)':'obesity_percentage'})
 
# making all the columns names with Upper case
columns_treatment(df)

df['OBESITY_PERCENTAGE']=df['OBESITY_PERCENTAGE'].str.split(' ').str[0]

#dropping nan values
df.dropna(inplace=True)

df = df[df["OBESITY_PERCENTAGE"]!='No']

df = df.astype({"OBESITY_PERCENTAGE": float})

print(df.dtypes)


create_table(df=df,tablename='OBESITY_PERCENTAGE',conn=conn )
df

COUNTRY_NAME           object
YEAR                    int64
OBESITY_PERCENTAGE    float64
SEX                    object
dtype: object


,COUNTRY_NAME,YEAR,OBESITY_PERCENTAGE,SEX
0,Afghanistan,1975,0.5,Both sexes
1,Afghanistan,1975,0.2,Male
2,Afghanistan,1975,0.8,Female
3,Afghanistan,1976,0.5,Both sexes
4,Afghanistan,1976,0.2,Male
...,...,...,...,...
24565,Zimbabwe,2015,4.5,Male
24566,Zimbabwe,2015,24.8,Female
24567,Zimbabwe,2016,15.5,Both sexes
24568,Zimbabwe,2016,4.7,Male


In [20]:
# Loading the data POPULATION
df = pd.read_csv('dataset\population.csv')

# melting the table
df = pd.melt(df.drop(['Indicator Code','Indicator Name'],axis=1), id_vars=['Country Name','Country Code'], value_vars=None, var_name='YEAR', value_name='total_population', col_level=None, ignore_index=True)

# making all the columns names with Upper case
columns_treatment(df)

#dropping nan values
df.dropna(inplace=True)

# changing the type of columns 
df = df.astype({"YEAR": int})

print(df.dtypes)
df.head(100)

create_table(df=df,tablename='POPULATION',conn=conn )
df

COUNTRY_NAME         object
COUNTRY_CODE         object
YEAR                  int32
TOTAL_POPULATION    float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,TOTAL_POPULATION
0,Aruba,ABW,1960,54208.0
1,Africa Eastern and Southern,AFE,1960,130836765.0
2,Afghanistan,AFG,1960,8996967.0
3,Africa Western and Central,AFW,1960,96396419.0
4,Angola,AGO,1960,5454938.0
...,...,...,...,...
16221,Kosovo,XKX,2020,1775378.0
16222,"Yemen, Rep.",YEM,2020,29825968.0
16223,South Africa,ZAF,2020,59308690.0
16224,Zambia,ZMB,2020,18383956.0


In [21]:
# Loading the data Rural Population
df = pd.read_csv(r'dataset\rural.csv')

# melting the table
df = pd.melt(df.drop(['Indicator Code','Indicator Name'],axis=1), id_vars=['Country Name','Country Code'], value_vars=None, var_name='YEAR', value_name='rural_population', col_level=None, ignore_index=True)

# making all the columns names with Upper case
columns_treatment(df)

#dropping nan values
df.dropna(inplace=True)

# changing the type of columns 
df = df.astype({"YEAR": int})

print(df.dtypes)
df.head(100)

create_table(df=df,tablename='RURAL',conn=conn )
df

COUNTRY_NAME         object
COUNTRY_CODE         object
YEAR                  int32
RURAL_POPULATION    float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,RURAL_POPULATION
0,Aruba,ABW,1960,49.224000
1,Africa Eastern and Southern,AFE,1960,85.295312
2,Afghanistan,AFG,1960,91.599000
3,Africa Western and Central,AFW,1960,85.329671
4,Angola,AGO,1960,89.565000
...,...,...,...,...
16220,Samoa,WSM,2020,82.111000
16222,"Yemen, Rep.",YEM,2020,62.092000
16223,South Africa,ZAF,2020,32.646000
16224,Zambia,ZMB,2020,55.371000


In [22]:
# Loading the data People Access Sanitation
df = pd.read_csv('dataset\sanitation.csv')

# melting the table
df = pd.melt(df.drop(['Indicator Code','Indicator Name'],axis=1), id_vars=['Country Name','Country Code'], value_vars=None, var_name='YEAR', value_name='people_sanitation', col_level=None, ignore_index=True)

# making all the columns names with Upper case
columns_treatment(df)

#dropping nan values
df.dropna(inplace=True)

# changing the type of columns 
df = df.astype({"YEAR": int})

print(df.dtypes)
df.head(100)

create_table(df=df,tablename='SANITATION',conn=conn )
df

COUNTRY_NAME          object
COUNTRY_CODE          object
YEAR                   int32
PEOPLE_SANITATION    float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,PEOPLE_SANITATION
10643,Africa Western and Central,AFW,2000,16.343739
10645,Albania,ALB,2000,40.237416
10646,Andorra,AND,2000,14.604036
10647,Arab World,ARB,2000,38.924729
10649,Argentina,ARG,2000,47.844621
...,...,...,...,...
16214,"Venezuela, RB",VEN,2020,23.066590
16219,World,WLD,2020,53.950146
16220,Samoa,WSM,2020,47.721437
16222,"Yemen, Rep.",YEM,2020,18.837979


In [23]:
# Loading the data Industrialization
df = pd.read_csv('dataset\Industrialization_intensity_index.csv')


df = df[df['Subindicator Type']=='Value']
df.rename(columns={'Country ISO3':'Country Code'},inplace=True)
 
# melting the table
df = pd.melt(df.drop(['Indicator Id','Subindicator Type'],axis=1), id_vars=['Country Name','Country Code', 'Indicator'], value_vars=None, var_name='YEAR', value_name='industrial_score'  )

#df = df.pivot(index=['Country Name','YEAR'], columns='industrial_score')

# making all the columns names with Upper case
columns_treatment(df)

#dropping nan values
df.dropna(inplace=True)

# changing the type of columns 
df = df.astype({"YEAR": int})

print(df.dtypes)
df.head(100)

df = df[df['INDICATOR']=='Competitive Industrial Performance Score']
df.drop('INDICATOR',axis=1,inplace=True)

create_table(df=df,tablename='INDUSTRIALIZATION',conn=conn) 
df

COUNTRY_NAME         object
COUNTRY_CODE         object
INDICATOR            object
YEAR                  int32
INDUSTRIAL_SCORE    float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,INDUSTRIAL_SCORE
0,Afghanistan,AFG,1990,0.005660
20,Angola,AGO,1990,0.002750
40,Albania,ALB,1990,0.008871
60,United Arab Emirates,ARE,1990,0.023798
80,Argentina,ARG,1990,0.067428
...,...,...,...,...
88060,Vietnam,VNM,2018,0.079977
88080,"Yemen, Rep.",YEM,2018,0.002871
88100,South Africa,ZAF,2018,0.056813
88120,Zambia,ZMB,2018,0.006345


In [24]:
# Loading the data Deaths by Cause
df = pd.read_csv('dataset\deaths_by_cause.csv')

# Renaming Columns
df = df.rename(columns={'Entity':'Country name','Code':'Country Code','Number of executions (Amnesty International)':'Executions'})

new_columns = []
for column in df.columns:
    try:
        new_column = column.split(' - ')[1]
    except IndexError:
        new_column = column
        if column == "Terrorism (deaths)":
            new_column = "Terrorism"
    new_columns.append(new_column)
df.columns = new_columns

 
# # melting the table
#df = pd.melt(df, id_vars=['Country','Country Code','Year'], value_vars=None, var_name='Cause', value_name='deaths_by_cause', col_level=None, ignore_index=True)


# drop nan
df = df.dropna()

# tratando nome das colunas
df = columns_treatment(df)

df = df[df['YEAR']>1960]



df = df.drop('EXECUTIONS',axis=1)
# verificando os tipos de variavel de cada coluna
print(df.dtypes)



# salvando 
create_table(df=df,tablename='DEATHS_BY_CAUSE',conn=conn )



 

df

COUNTRY_NAME                                   object
COUNTRY_CODE                                   object
YEAR                                            int64
MENINGITIS                                    float64
NEOPLASMS                                     float64
FIRE__HEAT__AND_HOT_SUBSTANCES                float64
MALARIA                                       float64
DROWNING                                      float64
INTERPERSONAL_VIOLENCE                        float64
HIV_AIDS                                      float64
DRUG_USE_DISORDERS                            float64
TUBERCULOSIS                                  float64
ROAD_INJURIES                                 float64
MATERNAL_DISORDERS                            float64
LOWER_RESPIRATORY_INFECTIONS                  float64
NEONATAL_DISORDERS                            float64
ALCOHOL_USE_DISORDERS                         float64
EXPOSURE_TO_FORCES_OF_NATURE                  float64
DIARRHEAL_DISEASES          

,COUNTRY_NAME,COUNTRY_CODE,YEAR,MENINGITIS,NEOPLASMS,FIRE__HEAT__AND_HOT_SUBSTANCES,MALARIA,DROWNING,INTERPERSONAL_VIOLENCE,HIV_AIDS,...,PROTEIN_ENERGY_MALNUTRITION,TERRORISM,CARDIOVASCULAR_DISEASES,CHRONIC_KIDNEY_DISEASE,CHRONIC_RESPIRATORY_DISEASES,CIRRHOSIS_AND_OTHER_CHRONIC_LIVER_DISEASES,DIGESTIVE_DISEASES,ACUTE_HEPATITIS,ALZHEIMERS_DISEASE_AND_OTHER_DEMENTIAS,PARKINSONS_DISEASE
0,Afghanistan,AFG,2007,2932.558591,15924.782541,481.219817,392.942607,2126.788535,3657.205757,148.276329,...,2438.567035,1199.0,53962.251859,4490.174240,7221.773068,3345.814979,6457.753476,3437.0,1402.0,450.0
1,Afghanistan,AFG,2008,2730.846137,16147.839026,462.180576,254.703403,1973.375893,3784.702895,157.401135,...,2230.538625,1092.0,54050.644196,4533.825139,7142.796656,3316.354776,6407.964590,3005.0,1424.0,455.0
2,Afghanistan,AFG,2009,2460.186423,16382.558546,448.300264,239.035635,1852.155239,3874.099828,167.270695,...,1997.531216,1065.0,53964.195557,4596.974422,7045.249090,3291.158847,6358.027600,2663.0,1449.0,460.0
3,Afghanistan,AFG,2011,2326.767573,17093.515668,447.884354,390.060248,1774.968476,4169.873038,183.816343,...,1805.467089,1525.0,54347.266593,4784.629776,6915.563606,3317.891662,6370.000383,2365.0,1508.0,473.0
4,Afghanistan,AFG,2012,2254.492968,17521.930189,445.226412,93.894201,1715.864705,4244.671108,190.525008,...,1666.898963,3521.0,54868.120368,4845.667028,6877.511217,3352.846566,6398.171770,2264.0,1544.0,482.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8486,Yemen,YEM,2011,536.588814,9437.185709,780.111826,4057.002894,857.725641,648.562886,261.760650,...,1392.583292,461.0,41647.934525,1996.802096,4538.515522,2422.575890,4193.685917,570.0,1282.0,302.0
8487,Yemen,YEM,2012,517.994823,9731.036653,761.465218,4409.525418,822.341430,661.269825,268.333457,...,1279.744519,1056.0,42823.727520,2055.232114,4620.024119,2416.772747,4205.852525,537.0,1337.0,315.0
8488,Yemen,YEM,2013,507.955660,10141.987867,777.017747,3955.396649,836.543551,672.426265,279.772375,...,1211.649089,624.0,44190.850256,2142.991494,4745.915146,2427.852121,4237.473169,517.0,1396.0,329.0
8489,Yemen,YEM,2014,499.350496,10462.258922,770.267425,4587.031995,820.602038,1136.097176,284.548011,...,1121.640703,1353.0,45318.119708,2214.893826,4825.280995,2419.483794,4230.676213,491.0,1453.0,343.0


In [25]:
# Loading the data Number of Deaths by Multiple Causes
df = pd.read_csv(r'dataset\number_of_deaths_by_multiple_causes.csv')

# Renaming Columns
df = df.rename(columns={'Entity':'Country name','Code':'Country Code'})

new_columns = []
for column in df.columns:
    try:
        new_column = column.split(' - ')[2]
        new_column = new_column.split(': ')[1]
    except IndexError:
        new_column = column
    new_columns.append(new_column)
df.columns = new_columns

# melting the table
#df = pd.melt(df, id_vars=['Country','Country Code','Year'], value_vars=None, var_name='Cause', value_name='deaths', col_level=None, ignore_index=True)

# drop nan
df = df.dropna()

# tratando nome das colunas
df = columns_treatment(df)

df = df[df['YEAR']>1960]

# verificando os tipos de variavel de cada coluna
print(df.dtypes)

# salvando 
create_table(df=df,tablename='DEATHS_BY_CAUSE',conn=conn )

#

df


COUNTRY_NAME                                 object
COUNTRY_CODE                                 object
YEAR                                          int64
OUTDOOR_AIR_POLLUTION                       float64
HIGH_SYSTOLIC_BLOOD_PRESSURE                float64
DIET_HIGH_IN_SODIUM                         float64
DIET_LOW_IN_WHOLE_GRAINS                    float64
ALCOHOL_USE                                 float64
DIET_LOW_IN_FRUITS                          float64
UNSAFE_WATER_SOURCE                         float64
SECONDHAND_SMOKE                            float64
LOW_BIRTH_WEIGHT                            float64
CHILD_WASTING                               float64
UNSAFE_SEX                                  float64
DIET_LOW_IN_NUTS_AND_SEEDS                  float64
HOUSEHOLD_AIR_POLLUTION_FROM_SOLID_FUELS    float64
DIET_LOW_IN_VEGETABLES                      float64
LOW_PHYSICAL_ACTIVITY                       float64
SMOKING                                     float64
HIGH_FASTING

,COUNTRY_NAME,COUNTRY_CODE,YEAR,OUTDOOR_AIR_POLLUTION,HIGH_SYSTOLIC_BLOOD_PRESSURE,DIET_HIGH_IN_SODIUM,DIET_LOW_IN_WHOLE_GRAINS,ALCOHOL_USE,DIET_LOW_IN_FRUITS,UNSAFE_WATER_SOURCE,...,HIGH_BODY_MASS_INDEX,UNSAFE_SANITATION,NO_ACCESS_TO_HANDWASHING_FACILITY,DRUG_USE,LOW_BONE_MINERAL_DENSITY,VITAMIN_A_DEFICIENCY,CHILD_STUNTING,DISCONTINUED_BREASTFEEDING,NON_EXCLUSIVE_BREASTFEEDING,IRON_DEFICIENCY
0,Afghanistan,AFG,1990,3169.846366,25633.129333,1044.908989,7077.315786,356.214705,3184.955042,3701.993900,...,9517.872973,2798.237588,4825.345186,173.578694,388.910751,2015.511499,7685.742698,106.901907,2216.041438,563.810676
1,Afghanistan,AFG,1991,3221.412133,25871.802088,1054.958389,7149.085364,363.730179,3248.376676,4309.282123,...,9489.185191,3254.498319,5127.178088,187.893678,388.784236,2056.353766,7885.672584,121.370213,2501.025200,610.788300
2,Afghanistan,AFG,1992,3394.426880,26308.794458,1074.605704,7297.308653,375.900229,3350.920662,5356.498055,...,9528.147750,4041.530409,5888.843950,210.813550,392.720873,2100.430812,8567.740635,150.417573,3052.538797,699.587347
3,Afghanistan,AFG,1993,3622.877918,26961.359541,1103.370486,7498.534388,388.571572,3479.811838,7151.520803,...,9611.335686,5391.672657,7006.908259,232.170925,410.670437,2315.590572,9875.290365,203.879604,3725.875633,772.889192
4,Afghanistan,AFG,1994,3787.712407,27658.423102,1133.882428,7697.588701,398.726977,3609.831607,7191.639304,...,9674.507905,5417.605900,7421.127808,247.296586,412.988840,2664.553808,11030.847296,204.444705,3832.531763,811.970644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8185,Zimbabwe,ZWE,2015,2835.033458,11483.307096,1062.771307,1353.591859,4853.718064,1820.364541,4335.906398,...,5635.604696,2878.905245,4328.086877,1067.765413,407.440849,137.892964,729.653732,20.996131,733.786878,281.981349
8186,Zimbabwe,ZWE,2016,2780.966656,11662.818042,1081.559177,1382.939523,4915.491728,1854.254697,4244.479623,...,5849.212698,2798.399984,4294.599669,1041.803465,414.933753,100.748641,674.072314,20.260844,699.988070,275.099504
8187,Zimbabwe,ZWE,2017,2699.958428,11818.670030,1097.734063,1408.916595,4991.753342,1882.975041,4192.616336,...,6046.518519,2744.133848,4250.682593,1007.374397,424.075515,82.429572,629.179174,19.324140,678.336881,266.913928
8188,Zimbabwe,ZWE,2018,2669.238506,12002.042218,1116.568032,1439.046279,5044.160520,1917.031590,4012.923127,...,6248.091739,2608.090663,4152.813064,969.354972,433.544694,84.654862,606.919388,17.213153,643.893170,260.991703


In [26]:
# Loading the data Deaths by Cause
df = pd.read_csv('dataset\substances-risk-factor-vs-direct-deaths.csv')

df = df.rename(columns={'Entity':'Country name',
                        'Code':'Country Code', 
                        'Deaths - Drug use disorders - Sex: Both - Age: All Ages (Number)':'Drug use disorders',
                        'Deaths - Alcohol use disorders - Sex: Both - Age: All Ages (Number)':'Alcohol use disorders',
                       'Deaths - Cause: All causes - Risk: Tobacco - Sex: Both - Age: All Ages (Number)':'Tobacco'})

df.drop(['Deaths - Cause: All causes - Risk: Drug use - Sex: Both - Age: All Ages (Number)',
        'Deaths - Cause: All causes - Risk: Alcohol use - Sex: Both - Age: All Ages (Number)'],axis=1,inplace=True)
'''
new_columns = []
for column in df.columns:
    try:
        new_column = column.split(' - ')[2]
        new_column = new_column.split(': ')[1]
    except IndexError:
        new_column = column
    new_columns.append(new_column)
df.columns = new_columns
'''



'''
df = pd.melt(df, id_vars=['Country','Country Code','Year'], value_vars=None, var_name='Risk Factor', value_name='deaths', col_level=None, ignore_index=True)
'''
# drop nan
df = df.dropna()

# tratando nome das colunas
df = columns_treatment(df)

df = df[df['YEAR']>1960]

# verificando os tipos de variavel de cada coluna
print(df.dtypes)

# salvando 
create_table(df=df,tablename='substances_riskfactor',conn=conn )

#

df

COUNTRY_NAME              object
COUNTRY_CODE              object
YEAR                       int64
DRUG_USE_DISORDERS       float64
ALCOHOL_USE_DISORDERS    float64
TOBACCO                  float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,DRUG_USE_DISORDERS,ALCOHOL_USE_DISORDERS,TOBACCO
0,Afghanistan,AFG,1990,93.444451,71.935533,9722.539324
1,Afghanistan,AFG,1991,102.382238,74.613084,9918.109213
2,Afghanistan,AFG,1992,117.856203,79.981038,10385.782432
3,Afghanistan,AFG,1993,132.343076,84.889252,10992.040309
4,Afghanistan,AFG,1994,142.180745,87.986117,11465.902846
...,...,...,...,...,...,...
8365,Zimbabwe,ZWE,2015,103.775228,47.926850,9862.272818
8366,Zimbabwe,ZWE,2016,109.677478,49.012089,9974.306189
8367,Zimbabwe,ZWE,2017,115.304607,49.942084,10059.969390
8368,Zimbabwe,ZWE,2018,121.004527,51.313963,10162.355845


In [27]:
# Loading the data Suicide Rate
df = pd.read_csv('dataset\suicide_rate.csv')

df = df.rename(columns={'Location':'Country Name','SpatialDimValueCode':'Country Code', 'ParentLocationCode':'Region Code', 'ParentLocation':'Region', 'Period': "Year", "Dim1": 'Sex', 'Value':'Suicides Range', 'FactValueNumeric':'Suicides', 'FactValueNumericLow':'SuicidesMin', 'FactValueNumericHigh':'SuicidesMax'})
df = df.drop(['IndicatorCode', 'Indicator', 'ValueType', 'Location type', 'Period type', 'IsLatestYear', 'Dim1 type', 'Dim1ValueCode', 'Language', 'DateModified'],axis=1)
# drop nan
df = df.dropna(axis=1, how='all')
df = df.dropna()

# tratando nome das colunas
df = columns_treatment(df)

df = df[df['YEAR']>1960]

# verificando os tipos de variavel de cada coluna
print(df.dtypes)

# salvando 
create_table(df=df,tablename='suicide_rate',conn=conn )

#


df

REGION_CODE        object
REGION             object
COUNTRY_CODE       object
COUNTRY_NAME       object
YEAR                int64
SEX                object
SUICIDES          float64
SUICIDESMIN       float64
SUICIDESMAX       float64
SUICIDES_RANGE     object
dtype: object


,REGION_CODE,REGION,COUNTRY_CODE,COUNTRY_NAME,YEAR,SEX,SUICIDES,SUICIDESMIN,SUICIDESMAX,SUICIDES_RANGE
0,AMR,Americas,ATG,Antigua and Barbuda,2019,Male,0.00,0.00,0.00,0 [0 – 0]
1,AMR,Americas,BRB,Barbados,2019,Female,0.16,0.11,0.22,0.16 [0.11 – 0.22]
2,AMR,Americas,BRB,Barbados,2019,Both sexes,0.31,0.22,0.42,0.31 [0.22 – 0.42]
3,AMR,Americas,ATG,Antigua and Barbuda,2019,Both sexes,0.32,0.22,0.45,0.32 [0.22 – 0.45]
4,AMR,Americas,BRB,Barbados,2019,Male,0.49,0.34,0.65,0.49 [0.34 – 0.65]
...,...,...,...,...,...,...,...,...,...,...
10975,AFR,Africa,GIN,Guinea,2000,Both sexes,9.73,5.57,15.96,9.73 [5.57 – 15.96]
10976,AFR,Africa,GHA,Ghana,2000,Both sexes,9.75,6.46,14.30,9.75 [6.46 – 14.3]
10977,EUR,Europe,MLT,Malta,2000,Male,9.75,7.56,12.29,9.75 [7.56 – 12.29]
10978,AFR,Africa,SYC,Seychelles,2000,Both sexes,9.76,6.60,13.46,9.76 [6.6 – 13.46]


In [28]:
# Loading the data Suicide Rate
df = pd.read_csv('dataset\suicide_rate.csv')

df = df.rename(columns={'Location':'Country Name','SpatialDimValueCode':'Country Code', 'ParentLocationCode':'Region Code', 'ParentLocation':'Region', 'Period': "Year", "Dim1": 'Sex', 'Value':'Suicides Range', 'FactValueNumeric':'Suicides', 'FactValueNumericLow':'SuicidesMin', 'FactValueNumericHigh':'SuicidesMax'})
df = df.drop(['IndicatorCode', 'Indicator', 'ValueType', 'Location type', 'Period type', 'IsLatestYear', 'Dim1 type', 'Dim1ValueCode', 'Language', 'DateModified'],axis=1)
# drop nan
df = df.dropna(axis=1, how='all')
df = df.dropna()

# tratando nome das colunas
df = columns_treatment(df)

df = df[df['YEAR']>1960]

df.drop(['REGION_CODE','REGION'],axis=1)

# verificando os tipos de variavel de cada coluna
print(df.dtypes)

# salvando 
create_table(df=df,tablename='suicide_rate',conn=conn )

#

df

REGION_CODE        object
REGION             object
COUNTRY_CODE       object
COUNTRY_NAME       object
YEAR                int64
SEX                object
SUICIDES          float64
SUICIDESMIN       float64
SUICIDESMAX       float64
SUICIDES_RANGE     object
dtype: object


,REGION_CODE,REGION,COUNTRY_CODE,COUNTRY_NAME,YEAR,SEX,SUICIDES,SUICIDESMIN,SUICIDESMAX,SUICIDES_RANGE
0,AMR,Americas,ATG,Antigua and Barbuda,2019,Male,0.00,0.00,0.00,0 [0 – 0]
1,AMR,Americas,BRB,Barbados,2019,Female,0.16,0.11,0.22,0.16 [0.11 – 0.22]
2,AMR,Americas,BRB,Barbados,2019,Both sexes,0.31,0.22,0.42,0.31 [0.22 – 0.42]
3,AMR,Americas,ATG,Antigua and Barbuda,2019,Both sexes,0.32,0.22,0.45,0.32 [0.22 – 0.45]
4,AMR,Americas,BRB,Barbados,2019,Male,0.49,0.34,0.65,0.49 [0.34 – 0.65]
...,...,...,...,...,...,...,...,...,...,...
10975,AFR,Africa,GIN,Guinea,2000,Both sexes,9.73,5.57,15.96,9.73 [5.57 – 15.96]
10976,AFR,Africa,GHA,Ghana,2000,Both sexes,9.75,6.46,14.30,9.75 [6.46 – 14.3]
10977,EUR,Europe,MLT,Malta,2000,Male,9.75,7.56,12.29,9.75 [7.56 – 12.29]
10978,AFR,Africa,SYC,Seychelles,2000,Both sexes,9.76,6.60,13.46,9.76 [6.6 – 13.46]


In [29]:
# Loading the data Under Five Deaths
df = pd.read_csv(r'dataset\number_of_under_five_deaths.csv')

df = df.rename(columns={'Location':'Country Name','SpatialDimValueCode':'Country Code', 'ParentLocationCode':'Region Code', 'ParentLocation':'Region', 'Period': "Year", "Dim1": 'Sex', 'Value':'u5 Deaths Range', 'FactValueNumeric':'u5 Deaths', 'FactValueNumericLow':'u5DeathsMin', 'FactValueNumericHigh':'u5DeathsMax'})
df = df.drop(['IndicatorCode', 'Indicator', 'ValueType', 'Location type', 'Period type', 'IsLatestYear', 'Dim1 type', 'Dim1ValueCode', 'Language', 'DateModified'],axis=1)
# drop nan
df = df.dropna(axis=1, how='all')
df = df.dropna()

# tratando nome das colunas
df = columns_treatment(df)

df = df[df['YEAR']>1960]

# verificando os tipos de variavel de cada coluna
print(df.dtypes)

df.drop(['REGION','REGION_CODE','U5DEATHSMIN','U5DEATHSMAX','U5_DEATHS_RANGE'], axis=1,inplace=True)

# salvando 
create_table(df=df,tablename='under5_deaths',conn=conn )


#

df

REGION_CODE        object
REGION             object
COUNTRY_CODE       object
COUNTRY_NAME       object
YEAR                int64
SEX                object
U5_DEATHS           int64
U5DEATHSMIN         int64
U5DEATHSMAX         int64
U5_DEATHS_RANGE    object
dtype: object


,COUNTRY_CODE,COUNTRY_NAME,YEAR,SEX,U5_DEATHS
0,SMR,San Marino,2019,Female,0
1,SMR,San Marino,2019,Male,0
2,NIU,Niue,2019,Male,0
3,SMR,San Marino,2019,Both sexes,0
4,MCO,Monaco,2019,Female,1
...,...,...,...,...,...
29833,TTO,Trinidad and Tobago,1961,Female,926
29834,GUY,Guyana,1961,Female,934
29835,NZL,New Zealand,1961,Male,939
29836,COL,Colombia,1961,Both sexes,95121


In [30]:
# Loading the data Estimates of Homicides
df = pd.read_csv(r'dataset\WHO_estimatesofhomicides.csv')

df = df.rename(columns={'Location':'Country Name','SpatialDimValueCode':'Country Code', 'ParentLocationCode':'Region Code', 'ParentLocation':'Region', 'Period': "Year", "Dim1": 'Sex', 'Value':'Homicides Range', 'FactValueNumeric':'Homicides estimates', 'FactValueNumericLow':'homicidesMin', 'FactValueNumericHigh':'homicidesMax'})
df = df.drop(['IndicatorCode', 'Indicator', 'ValueType', 'Location type', 'Period type', 'IsLatestYear', 'Dim1 type', 'Dim1ValueCode', 'Language', 'DateModified'],axis=1)
# drop nan
df = df.dropna(axis=1, how='all')
df = df.dropna()

# tratando nome das colunas
df = columns_treatment(df)

df = df[df['YEAR']>1960]

# verificando os tipos de variavel de cada coluna
print(df.dtypes)

df.drop(['HOMICIDESMIN','HOMICIDESMAX','HOMICIDES_RANGE','REGION_CODE','REGION'],axis=1,inplace=True)

# salvando 
create_table(df=df,tablename='homicides_estimates',conn=conn )

#

df

REGION_CODE             object
REGION                  object
COUNTRY_CODE            object
COUNTRY_NAME            object
YEAR                     int64
SEX                     object
HOMICIDES_ESTIMATES    float64
HOMICIDESMIN           float64
HOMICIDESMAX           float64
HOMICIDES_RANGE         object
dtype: object


,COUNTRY_CODE,COUNTRY_NAME,YEAR,SEX,HOMICIDES_ESTIMATES
0,ATG,Antigua and Barbuda,2019,Female,0.61
1,BRN,Brunei Darussalam,2019,Female,1.00
2,CAF,Central African Republic,2019,Both sexes,1009.00
3,PRK,Democratic People's Republic of Korea,2019,Both sexes,1082.00
4,BOL,Bolivia (Plurinational State of),2019,Both sexes,1097.00
...,...,...,...,...,...
10975,HUN,Hungary,2000,Female,97.01
10976,LVA,Latvia,2000,Female,99.00
10977,MRT,Mauritania,2000,Female,99.09
10978,CHL,Chile,2000,Female,99.82


In [31]:
# Loading the data Mean Body Mass Index
dfmales = pd.read_csv('dataset\Mean-body-mass-index-bmi-in-adult-males.csv')
dfwomen =  pd.read_csv('dataset\Mean-body-mass-index-bmi-in-adult-women.csv')

df = pd.merge(dfmales, dfwomen, on = ['Entity','Code','Year'])

df = df.rename(columns={'Entity':'Country Name','Code':'Country Code', 'Mean BMI (male)': 'bmi_male', 'Mean BMI (female)': 'bmi_female'})
# drop nan
df = df.dropna(axis=1, how='all')
df = df.dropna()

# tratando nome das colunas
df = columns_treatment(df)

df = df[df['YEAR']>1960]

# verificando os tipos de variavel de cada coluna
print(df.dtypes)

# salvando 
create_table(df=df,tablename='mean_bmi',conn=conn )

#

df

COUNTRY_NAME     object
COUNTRY_CODE     object
YEAR              int64
BMI_MALE        float64
BMI_FEMALE      float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,BMI_MALE,BMI_FEMALE
0,Afghanistan,AFG,1975,18.999440,18.844326
1,Afghanistan,AFG,1976,19.105518,18.986417
2,Afghanistan,AFG,1977,19.212080,19.128086
3,Afghanistan,AFG,1978,19.319053,19.269247
4,Afghanistan,AFG,1979,19.425952,19.409477
...,...,...,...,...,...
8815,Zimbabwe,ZWE,2012,22.181020,25.284840
8816,Zimbabwe,ZWE,2013,22.209511,25.319948
8817,Zimbabwe,ZWE,2014,22.239100,25.356222
8818,Zimbabwe,ZWE,2015,22.269522,25.393409


In [32]:
# Loading the data Air Pollution
df = pd.read_csv(r'dataset\Pastas\theworldbank_airpollution\API_EN.ATM.PM25.MC.ZS_DS2_en_csv_v2_3481084.csv')

# drop das colunas que nao queremos
df = df.drop(['Indicator Code','Indicator Name'], axis=1)

# melt
df = pd.melt(df, id_vars=['Country Name','Country Code'], value_vars=None, var_name='YEAR', value_name='Percentage of People Exposed to Air Pollution', col_level=None, ignore_index=True)

# drop nan
df = df.dropna()

# tratando nome das colunas
df = columns_treatment(df)

# verificando os tipos de variavel de cada coluna
print(df.dtypes)

# Mudando a coluna ano para INT
df = df.astype({"YEAR": int})

# # verificando os tipos de variavel de cada coluna
print(df.dtypes)

# # salvando 
create_table(df=df,tablename='airpollution',conn=conn )

df

COUNTRY_NAME                                      object
COUNTRY_CODE                                      object
YEAR                                              object
PERCENTAGE_OF_PEOPLE_EXPOSED_TO_AIR_POLLUTION    float64
dtype: object
COUNTRY_NAME                                      object
COUNTRY_CODE                                      object
YEAR                                               int32
PERCENTAGE_OF_PEOPLE_EXPOSED_TO_AIR_POLLUTION    float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,PERCENTAGE_OF_PEOPLE_EXPOSED_TO_AIR_POLLUTION
7981,Africa Eastern and Southern,AFE,1990,100.0
7982,Afghanistan,AFG,1990,100.0
7983,Africa Western and Central,AFW,1990,100.0
7984,Angola,AGO,1990,100.0
7985,Albania,ALB,1990,100.0
...,...,...,...,...
15422,Samoa,WSM,2017,100.0
15424,"Yemen, Rep.",YEM,2017,100.0
15425,South Africa,ZAF,2017,100.0
15426,Zambia,ZMB,2017,100.0


In [33]:
# Loading the data CO2 emissions
df = pd.read_csv(r'dataset\Pastas\theworldbank_co2emissions\API_EN.ATM.CO2E.KT_DS2_en_csv_v2_3470002.csv')

# drop das colunas que nao queremos
df = df.drop(['Indicator Code','Indicator Name'], axis=1)

# melt
df = pd.melt(df, id_vars=['Country Name','Country Code'], value_vars=None, var_name='YEAR', value_name='C02 emissions', col_level=None, ignore_index=True)

# drop nan
df = df.dropna()

# tratando nome das colunas
df = columns_treatment(df)

# verificando os tipos de variavel de cada coluna
print(df.dtypes)

# Mudando a coluna ano para INT
df = df.astype({"YEAR": int})

# # verificando os tipos de variavel de cada coluna
print(df.dtypes)

# # salvando 
create_table(df=df,tablename='co2emissions',conn=conn )

df

COUNTRY_NAME      object
COUNTRY_CODE      object
YEAR              object
C02_EMISSIONS    float64
dtype: object
COUNTRY_NAME      object
COUNTRY_CODE      object
YEAR               int32
C02_EMISSIONS    float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,C02_EMISSIONS
0,Aruba,ABW,1960,11092.6750
1,Africa Eastern and Southern,AFE,1960,118545.9013
2,Afghanistan,AFG,1960,414.3710
3,Africa Western and Central,AFW,1960,8760.4630
4,Angola,AGO,1960,550.0500
...,...,...,...,...
15688,Samoa,WSM,2018,320.0000
15690,"Yemen, Rep.",YEM,2018,9310.0000
15691,South Africa,ZAF,2018,433250.0000
15692,Zambia,ZMB,2018,7740.0000


In [34]:
# Loading the data Corruption From 1 to 6
df = pd.read_csv(r'dataset\Pastas\theworldbank_transparencycorruption\API_IQ.CPA.TRAN.XQ_DS2_en_csv_v2_3469422.csv')

# drop das colunas que nao queremos
df = df.drop(['Indicator Code','Indicator Name'], axis=1)

# melt
df = pd.melt(df, id_vars=['Country Name','Country Code'], value_vars=None, var_name='YEAR', value_name='CPIA corruption in the public sector from 1 to 6', col_level=None, ignore_index=True)

# drop nan
df = df.dropna()

# tratando nome das colunas
df = columns_treatment(df)

# verificando os tipos de variavel de cada coluna
print(df.dtypes)

# Mudando a coluna ano para INT
df = df.astype({"YEAR": int})

# # verificando os tipos de variavel de cada coluna
print(df.dtypes)

# # salvando 
create_table(df=df,tablename='cpia_corruption',conn=conn )

df

COUNTRY_NAME                                         object
COUNTRY_CODE                                         object
YEAR                                                 object
CPIA_CORRUPTION_IN_THE_PUBLIC_SECTOR_FROM_1_TO_6    float64
dtype: object
COUNTRY_NAME                                         object
COUNTRY_CODE                                         object
YEAR                                                  int32
CPIA_CORRUPTION_IN_THE_PUBLIC_SECTOR_FROM_1_TO_6    float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,CPIA_CORRUPTION_IN_THE_PUBLIC_SECTOR_FROM_1_TO_6
11971,Africa Eastern and Southern,AFE,2005,2.805556
11973,Africa Western and Central,AFW,2005,2.789474
11974,Angola,AGO,2005,2.500000
11975,Albania,ALB,2005,3.000000
11977,Arab World,ARB,2005,2.500000
...,...,...,...,...
16220,Samoa,WSM,2020,4.000000
16221,Kosovo,XKX,2020,3.000000
16222,"Yemen, Rep.",YEM,2020,1.500000
16224,Zambia,ZMB,2020,2.500000


In [35]:
# Loading the data Urban Population
df = pd.read_csv(r'dataset\Pastas\theworldbank_urbanpopulation\API_SP.URB.TOTL_DS2_en_csv_v2_3477498.csv')

# drop das colunas que nao queremos
df = df.drop(['Indicator Code','Indicator Name'], axis=1)

# melt
df = pd.melt(df, id_vars=['Country Name','Country Code'], value_vars=None, var_name='YEAR', value_name='Urban Population', col_level=None, ignore_index=True)

# drop nan
df = df.dropna()

# tratando nome das colunas
df = columns_treatment(df)

# verificando os tipos de variavel de cada coluna
print(df.dtypes)

# Mudando a coluna ano para INT
df = df.astype({"YEAR": int})

# # verificando os tipos de variavel de cada coluna
print(df.dtypes)

# # salvando 
create_table(df=df,tablename='urban_population',conn=conn )

df

COUNTRY_NAME         object
COUNTRY_CODE         object
YEAR                 object
URBAN_POPULATION    float64
dtype: object
COUNTRY_NAME         object
COUNTRY_CODE         object
YEAR                  int32
URBAN_POPULATION    float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,URBAN_POPULATION
0,Aruba,ABW,1960,27525.0
1,Africa Eastern and Southern,AFE,1960,19239140.0
2,Afghanistan,AFG,1960,755835.0
3,Africa Western and Central,AFW,1960,14141671.0
4,Angola,AGO,1960,569223.0
...,...,...,...,...
16220,Samoa,WSM,2020,35494.0
16222,"Yemen, Rep.",YEM,2020,11306428.0
16223,South Africa,ZAF,2020,39946775.0
16224,Zambia,ZMB,2020,8204576.0


In [36]:
# Loading the data Literacy Rate
df = pd.read_csv(r'dataset\Literacy rate, adult total (% of people ages 15 and above ).csv')

# drop das colunas que nao queremos
df = df.drop(['Indicator Code','Indicator Name'], axis=1)

# melt
df = pd.melt(df, id_vars=['Country Name','Country Code'], value_vars=None, var_name='YEAR', value_name='Literacy Rate', col_level=None, ignore_index=True)

# drop nan
df = df.dropna()

# tratando nome das colunas
df = columns_treatment(df)

# verificando os tipos de variavel de cada coluna
print(df.dtypes)

# Mudando a coluna ano para INT
df = df.astype({"YEAR": int})

# # verificando os tipos de variavel de cada coluna
print(df.dtypes)

# # salvando 
create_table(df=df,tablename='literacy_rate',conn=conn )

df

COUNTRY_NAME      object
COUNTRY_CODE      object
YEAR              object
LITERACY_RATE    float64
dtype: object
COUNTRY_NAME      object
COUNTRY_CODE      object
YEAR               int32
LITERACY_RATE    float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,LITERACY_RATE
2690,Barbados,BRB,1970,99.268082
2750,Grenada,GRD,1970,97.794472
2913,St. Vincent and the Grenadines,VCT,1970,95.632156
3164,Middle East & North Africa (IDA & IBRD countries),TMN,1971,38.724621
3353,Middle East & North Africa (excluding high inc...,MNA,1972,39.562611
...,...,...,...,...
16198,Middle East & North Africa (IDA & IBRD countries),TMN,2020,76.476593
16200,South Asia (IDA & IBRD),TSA,2020,73.654518
16201,Sub-Saharan Africa (IDA & IBRD countries),TSS,2020,65.858719
16209,Upper middle income,UMC,2020,95.796822


In [3]:
# creating the final table

df = pd.read_sql("SELECT * FROM LIFE_EXPECTANCY;", conn)

#df_final = mg("SELECT * FROM ADULT_MORTALITY;",conn,df)
df_final = mg("SELECT * FROM airpollution;",conn,df)
df_final = mg("SELECT * FROM co2emissions;",conn,df_final)
df_final = mg("SELECT * FROM cpia_corruption;",conn,df_final)
df_final = mg("SELECT * FROM daily_supply_of_calories;",conn,df_final)
df_final = mg("SELECT * FROM deaths_by_cause;",conn,df_final)
df_final = mg("SELECT * FROM electricity;",conn,df_final)
df_final = mg("SELECT * FROM gdp;",conn,df_final)
df_final = mg("SELECT * FROM homicides_estimates;",conn,df_final)
df_final = mg("SELECT * FROM literacy_rate;",conn,df_final)
df_final = mg("SELECT * FROM mean_bmi;",conn,df_final)
df_final = mg("SELECT * FROM overweight_in_children;",conn,df_final)
df_final = mg("SELECT * FROM population;",conn,df_final)
df_final = mg("SELECT * FROM rural;",conn,df_final)
df_final = mg("SELECT * FROM sanitation;",conn,df_final)
df_final = mg("SELECT * FROM substances_riskfactor;",conn,df_final)
df_final = mg("SELECT * FROM suicide_rate;",conn,df_final)
#df_final = df_final.drop(['drug_use_disorders_y','alcohol_use_disorders_y'],axis=1)
df_final = mg("SELECT * FROM under5_deaths;",conn,df_final)
df_final = mg("SELECT * FROM urban_population;",conn,df_final)
df_final = mg("SELECT * FROM obesity_percentage;",conn,df_final)

df_final= df_final.drop(['suicides_range','region','region_code'],axis=1)

#

create_table(df=df_final,tablename='final_table',conn=conn )


In [4]:
df_final = df_final[df_final['country_name']=='Brazil']


In [5]:
df_final.columns

Index(['country_name', 'country_code', 'year', 'life_expectancy',
       'percentage_of_people_exposed_to_air_pollution', 'c02_emissions',
       'cpia_corruption_in_the_public_sector_from_1_to_6',
       'daily_caloric_supply', 'overweight_or_obese', 'outdoor_air_pollution',
       'high_systolic_blood_pressure', 'diet_high_in_sodium',
       'diet_low_in_whole_grains', 'alcohol_use', 'diet_low_in_fruits',
       'unsafe_water_source', 'secondhand_smoke', 'low_birth_weight',
       'child_wasting', 'unsafe_sex', 'diet_low_in_nuts_and_seeds',
       'household_air_pollution_from_solid_fuels', 'diet_low_in_vegetables',
       'low_physical_activity', 'smoking', 'high_fasting_plasma_glucose',
       'air_pollution', 'high_body_mass_index', 'unsafe_sanitation',
       'no_access_to_handwashing_facility', 'drug_use',
       'low_bone_mineral_density', 'vitamin_a_deficiency', 'child_stunting',
       'discontinued_breastfeeding', 'non_exclusive_breastfeeding',
       'iron_deficiency', 'acc

In [ ]:
df_final.dtypes

In [6]:
conn.close()